<a href="https://colab.research.google.com/github/41371130H/PL-Repo/blob/main/HW6_%E7%B0%A1%E5%A0%B1%E7%AD%86%E8%A8%98%E8%88%87%E5%80%8B%E4%BA%BA%E7%9F%A5%E8%AD%98%E5%BA%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 AI 簡報筆記知識庫 - 介紹與操作手冊

**您不該只是「儲存」筆記，您應該要能與您的筆記「互動」**。

本系統能讀懂您在簡報 PDF 上的「原文」以及您「手寫的筆記」，AI 會自動分析這兩者之間的關聯，幫您提煉出「您當時在想什麼」，並將這些洞察轉化為一個可供搜尋、可供分析的個人化知識庫。

### 🌟 核心功能

* **智慧分析**：同時讀取簡報內文與您的手寫筆記，自動生成課程摘要、筆記洞察與建議行動。
* **永久儲存**：自動將每一次的分析結果，結構化地存入您的 Google Sheet (`Knowledge_Base`)，建立您的「第二大腦」。
* **即時問答**：針對您「剛剛上傳」的檔案，您可以立即在 Chatbot 中追問細節。
* **全局總覽**：一鍵切換到「歷史知識庫」，瀏覽您所有的歷史筆記。
* **全局分析**：AI 能讀取您**所有**的歷史筆記，自動生成一份「學習成效分析報告」(包含您的學習時段、主題、知識缺漏)，並將報告存回 Google Sheet (`Gemini 全局分析`)。
* **全局問答**：您可以對「AI 知識庫總管」提問，AI 會搜尋您**過去所有**的筆記，並給您一份綜合回答。

---

### 🖥️ 介面操作指南

Gradio 介面分為兩大區塊：左側的「上傳區」與右側的「分頁顯示區」。

#### 1. 左側：上傳與啟動

* **上傳您的筆記簡報 PDF**：點擊此處來上傳您「已寫上筆記」的簡報 PDF 檔案。
* **🚀 開始分析並存入知識庫**：上傳檔案後，點擊此按鈕。
* **狀態欄**：按鈕下方會顯示目前的處理狀態 (例如：「🤖 正在向 Gemini 2.5 Pro 提出請求...」或「✅ 分析並儲存完畢！」)。

#### 2. 右側：分析結果與知識庫

這部分有兩個主要分頁：

#### A. 🎓 分頁一：新資料分析

> **用途**：專注於您「剛剛上傳」的這份檔案。

這個分頁內部又有三個「子分頁」：

* **子分頁 (1) - 課程簡報摘要**：
    * AI 從簡報中提取的**客觀資訊**。
    * 包含：`課程名稱`、`簡報主題`、`簡報內容摘要`。

* **子分頁 (2) - 我的筆記洞察**：
    * **AI 綜合分析**您的筆記後，得出的**個人化洞察** (這是最有價值的部分)。
    * 包含：`筆記摘要` (AI 總結您寫了什麼)、`筆記洞察` (AI 分析您的筆記意圖)、`建議行動` (AI 建議您下一步該做什麼)。

* **子分頁 (3) - AI 提問**：
    * 這是一個**「針對此檔案」**的聊天機器人。
    * 您可以針對這份簡報的任何細節提問，例如：「請幫我詳細說明『筆記洞察』中的第二點」。

#### B. 📚 分頁二：歷史知識庫

> **用途**：管理和分析您「所有」的歷史筆記。

這個分頁內部同樣有三個「子分頁」：

* **子分頁 (1) - 資料總表**：
    * **Google Sheet (`Knowledge_Base`) 的即時鏡像**。
    * 您所有上傳過的歷史紀錄都會以表格形式顯示在這裡，方便您快速瀏覽。

* **子分頁 (2) - Gemini 全局分析**：
    * **您的個人 AI 學習教練**。
    * 點擊「**🚀 開始進行全局學習分析**」按鈕。
    * AI 會讀取「資料總表」中的**所有資料**，分析您的學習習慣 (例如：您最常在晚上學習、您對「演算法」相關主題特別有疑問) 和知識缺漏。
    * 分析報告會顯示在下方，並**自動儲存**到 Google Sheet 的 `Gemini 全局分析` 分頁中，供您日後追蹤。

* **子分頁 (3) - AI 知識庫總管**：
    * **您的「第二大腦」搜尋引擎**。
    * 這是一個**「全局」**聊天機器人。
    * 您可以問它**跨越所有檔案**的問題，例如：「我過去所有關於「機器學習」的筆記洞察有哪些？」或「我上個月學了哪些主題？」。

In [66]:
!pip install --upgrade gradio

In [67]:
# === 1. 基本套件 ===
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
import pytz # 用於設定時區
import gradio as gr # 導入 Gradio

# === 2. Google Colab 相關 ===
from google.colab import auth, files, userdata
from google.auth import default

# === 3. Google AI (Gemini) ===
import google.generativeai as genai

# === 4. Google 帳號授權 (Colab) ===
print(" authorizing Google account...")
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("✅ Google 帳號授權完成。")
except Exception as e:
    print(f"❌ 授權失敗: {e}")
    print("請確保您已允許 Colab 存取您的 Google 帳號。")


# === 5. Google Sheet 參數設定 (!!!) ===
SHEET_URL = "https://docs.google.com/spreadsheets/d/1kIjMhUpcFe-MoYmL2bvjJ_4lf6UT-o3zXF7F82kFmfg/edit?usp=sharing"
WORKSHEET_NAME = "Knowledge_Base"
ANALYSIS_WORKSHEET_NAME = "Gemini 全局分析"

print(f"將使用 Google Sheet: {SHEET_URL}")
print(f"將寫入分頁: {WORKSHEET_NAME}")

 authorizing Google account...
✅ Google 帳號授權完成。
將使用 Google Sheet: https://docs.google.com/spreadsheets/d/1kIjMhUpcFe-MoYmL2bvjJ_4lf6UT-o3zXF7F82kFmfg/edit?usp=sharing
將寫入分頁: Knowledge_Base


In [68]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
# (請確認您已在 Colab 的「密鑰」中儲存了 'gemini' 這個名稱)
try:
    api_key = userdata.get('gemini')
    genai.configure(api_key=api_key)
    print("✅ Gemini API Key 設定完成。")
except userdata.SecretNotFoundError:
    print("❌ 錯誤：找不到名稱為 'gemini' 的 Colab 密鑰。")
    print("請點選左側「鑰匙 🔑」圖示，新增一個密鑰，名稱設為 'gemini'，值設為您的 API Key。")
except Exception as e:
    print(f"❌ API Key 設定時發生錯誤: {e}")

✅ Gemini API Key 設定完成。


In [69]:
# === 1. 輔助函式：從 Google Sheet 載入資料 ===
# (此函式會在 Gradio 啟動和每次更新後被呼叫)
def load_knowledge_base():
    """
    從 Google Sheet 讀取 'Knowledge_Base' 分頁的現有資料。
    使用新的中文欄位名稱。
    """
    print("正在從 Google Sheet 載入資料...")
    try:
        gsheets = gc.open_by_url(SHEET_URL)
        wk = gsheets.worksheet(WORKSHEET_NAME)
        data = wk.get_all_records()
        if not data:
            print("知識庫分頁為空。")
            # 回傳一個空的 DataFrame，但包含我們預期的中文欄位
            cols = ["課程名稱", "簡報主題", "簡報內容摘要", "筆記摘要", "筆記洞察", "建議行動", "上傳時間"]
            return pd.DataFrame(columns=cols)

        df = pd.DataFrame(data)
        # 確保所有欄位都存在，避免 Concat 失敗
        expected_cols = ["課程名稱", "簡報主題", "簡報內容摘要", "筆記摘要", "筆記洞察", "建議行動", "上傳時間"]
        for col in expected_cols:
            if col not in df.columns:
                df[col] = pd.NA
        print(f"✅ 成功載入 {len(df)} 筆資料。")
        return df[expected_cols] # 確保欄位順序正確
    except gspread.exceptions.WorksheetNotFound:
        print(f"找不到分頁 '{WORKSHEET_NAME}'，將嘗試建立...")
        try:
            gsheets = gc.open_by_url(SHEET_URL)
            wk = gsheets.add_worksheet(title=WORKSHEET_NAME, rows="100", cols="20")
            # 建立分頁後，寫入標頭
            headers = ["課程名稱", "簡報主題", "簡報內容摘要", "筆記摘要", "筆記洞察", "建議行動", "上傳時間"]
            wk.append_row(headers)
            print(f"✅ 已建立新分頁 '{WORKSHEET_NAME}' 並寫入標頭。")
            return pd.DataFrame(columns=headers)
        except Exception as e:
            print(f"❌ 建立分頁時出錯: {e}")
            return pd.DataFrame()
    except Exception as e:
        print(f"❌ 載入 Google Sheet 資料時出錯: {e}")
        return pd.DataFrame()

In [70]:
# === 2. 核心函式：處理上傳的筆記 ===
# (此函式會被 Gradio 按鈕觸發)
def process_new_note(pdf_file, current_file_name_state, progress=gr.Progress(track_tqdm=True)):
    """
    接收 Gradio 上傳的 PDF，分析，寫入 Sheet，並回傳分析結果和檔案名稱。
    """
    if pdf_file is None:
        return "⚠️ 請先上傳一個 PDF 檔案。", None, None, None, None, None, None, [], [], current_file_name_state, load_knowledge_base()

    try:
        pdf_filename = pdf_file.name

        # --- 步驟 1: 上傳 PDF 至 Gemini ---
        progress(0.1, desc="正在上傳 PDF 至 Gemini File API...")
        print(f"正在處理檔案: '{pdf_filename}'")
        gemini_file = genai.upload_file(path=pdf_filename,
                                        display_name=f"Annotated Slides: {os.path.basename(pdf_filename)}")
        new_gemini_file_name = gemini_file.name # 儲存新檔案的名稱
        print(f"✅ 檔案上傳成功！(File Name: {new_gemini_file_name})")

        # --- 步驟 1.5: 刪除「上一個」暫存檔案 (如果存在) ---
        if current_file_name_state:
            try:
                print(f"正在清理上一個暫存檔案: {current_file_name_state}")
                genai.delete_file(current_file_name_state)
                print("✅ 上一個檔案清理完畢。")
            except Exception as del_e:
                print(f"⚠️ 清理舊檔案 {current_file_name_state} 時出錯 (可能已被刪除): {del_e}")

        # --- 步驟 2: 準備 Prompt (!!! 使用新的中文欄位 !!!) ---
        progress(0.3, desc="正在準備 AI Prompt...")
        tw_tz = pytz.timezone('Asia/Taipei')
        now_str = dt.now(tw_tz).strftime("%Y-%m-%d %H:%M:%S")

        prompt_text = f"""
        你是一位頂尖的 AI 學習助理。
        請分析我上傳的 PDF 簡報檔案 (檔案名稱: {pdf_filename})。
        這個檔案同時包含了老師的簡報原文，以及我個人的「筆記」(手寫、劃線、註解)。
        你的任務是「綜合」這兩者，幫我提煉出一個結構化的學習紀錄。

        你的輸出「必須」是 JSON 格式，「只」輸出 JSON 內容，不要有任何其他說明文字。

        JSON 格式的欄位必須如下 (使用繁體中文作為 key)：
        {{
          "課程名稱": "（AI 推測的課程名稱）",
          "簡報主題": "（AI 推測的本週簡報主題）",
          "簡報內容摘要": "（AI 總結的「老師簡報內容」摘要）",
          "筆記摘要": "（AI 總結的「我的筆記」摘要）",
          "筆記洞察": "（AI 根據我的筆記，提煉出的 1-3 個關鍵洞察、疑問、或重點，請用 \\n 分隔）",
          "建議行動": "（AI 建議的下一步行動，例如：複習 'Elbow method'，請用 \\n 分隔）",
          "上傳時間": "{now_str}"
        }}
        """

        # --- 步驟 3: 呼叫 Gemini 2.5 Pro ---
        progress(0.5, desc="🤖 正在向 Gemini 2.5 Pro 提出請求 (這可能需要一點時間)...")
        model = genai.GenerativeModel('gemini-2.5-pro')
        response = model.generate_content([prompt_text, gemini_file])

        # --- (注意：我們暫時不刪除 gemini_file，以便 Chatbot 使用) ---

        # --- 步驟 5: 解析 JSON ---
        json_output = response.text.strip().replace("```json\n", "").replace("```", "")
        data_dict = json.loads(json_output)

        # 確保所有 key 都存在，若 AI 遺漏則填入空值
        expected_keys = ["課程名稱", "簡報主題", "簡報內容摘要", "筆記摘要", "筆記洞察", "建議行動"]
        for key in expected_keys:
            if key not in data_dict:
                data_dict[key] = f"AI 未提供 '{key}'"
        data_dict["上傳時間"] = now_str # 確保時間正確

        df_new_row = pd.DataFrame([data_dict])
        print("✅ AI 分析完成，已解析 JSON。")

        # --- 步驟 6: 寫入 Google Sheet ---
        progress(0.8, desc="正在將 AI 分析結果寫入 Google Sheet...")
        gsheets = gc.open_by_url(SHEET_URL)
        wk = gsheets.worksheet(WORKSHEET_NAME)

        # 讀取現有資料 (DataFrame 格式)
        df_existing = load_knowledge_base()

        # 合併新舊資料
        df_out = pd.concat([df_existing, df_new_row], ignore_index=True)

        # 寫回 Google Sheet
        set_with_dataframe(wk, df_out)
        print("✅ 已將新資料附加到 Google Sheet。")

        # --- 步驟 7: 回傳所有結果給 Gradio 介面 ---
        progress(1.0, desc="✅ 分析並儲存完畢！")

        # 重新載入一次以確保資料最新
        df_updated = load_knowledge_base()

        return (
            "✅ 分析並儲存完畢！請在右方查看結果，或針對此檔案提問。",
            data_dict["課程名稱"],
            data_dict["簡報主題"],
            data_dict["簡報內容摘要"],
            data_dict["筆記摘要"],
            data_dict["筆記洞察"],
            data_dict["建議行動"],
            [], # 清空 Chatbot 顯示
            [], # 清空 Chatbot 歷史
            new_gemini_file_name, # 儲存「新」檔案的名稱
            df_updated # 更新歷史資料表格
        )

    except Exception as e:
        print(f"❌ 處理過程中發生錯誤：{e}")
        # 清理剛上傳失敗的檔案
        if 'gemini_file' in locals():
            try:
                genai.delete_file(gemini_file.name)
                print("已清理上傳失敗的檔案。")
            except Exception as del_e:
                print(f"清理失敗檔案時出錯: {del_e}")

        return f"❌ 執行時發生錯誤：{e}", None, None, None, None, None, None, [], [], current_file_name_state, load_knowledge_base()

In [71]:
# === 3. Chatbot 函式 ===
def handle_chat(message, history, file_name):
    """
    處理 Chatbot 的問答，會基於儲存的 file_name 來提問。
    """
    if not file_name:
        history.append((message, "❌ 錯誤：請先上傳並成功分析一個檔案，才能開始提問。"))
        return "", history

    try:
        print(f"Chatbot 正在使用檔案: {file_name}")
        # 顯示使用者的問題
        history.append((message, None))

        # 準備呼叫 Gemini
        model = genai.GenerativeModel('gemini-2.5-pro')

        # 從 file_name 取得檔案 (這一步是關鍵)
        gemini_file = genai.get_file(file_name)

        # 建立 Chat 歷史 (簡易版，只傳送目前問題和檔案)
        # (更進階的作法是將 history 轉換為 Gemini 的格式)
        chat_prompt = f"""
        你是一位 AI 學習助理。請根據我上傳的檔案 (已附上) 和我的問題來回答。
        請專注回答我的問題，保持簡潔。

        [我的問題]:
        {message}
        """

        # 呼叫 AI
        response = model.generate_content([chat_prompt, gemini_file])

        # 更新 Chatbot 歷史
        history[-1] = (message, response.text)

        return "", history

    except Exception as e:
        print(f"❌ Chatbot 處理時發生錯誤：{e}")
        history.append((message, f"❌ 處理時發生錯誤：{e}"))
        return "", history

In [72]:
# === 3.A 全局分析函式 (for 歷史分頁) ===
def analyze_knowledge_base(progress=gr.Progress(track_tqdm=True)):
    """
    讀取整個 Google Sheet，並呼叫 Gemini 進行全局分析。
    """
    progress(0.1, desc="正在從 Google Sheet 載入所有歷史資料...")
    print("全局分析：正在載入資料...")
    try:
        df = load_knowledge_base()
        if df.empty:
            return "知識庫中尚無資料可供分析。"

        # 將 DataFrame 轉換為 Gemini 易讀的 Markdown 格式
        data_markdown = df.to_markdown(index=False)
        progress(0.4, desc="資料載入完畢，正在準備 AI Prompt...")

        # 準備 Prompt
        prompt_text = f"""
        你是一位頂尖的 AI 學習助理與資料分析師。
        我將提供你我完整的「簡報筆記知識庫」（Markdown 格式）。
        請你根據「所有」資料，幫我進行一次全面的學習成效分析。

        你的分析應包含以下面向：
        1.  **學習主題分佈**：我最常學習的「課程名稱」或「簡報主題」是什麼？
        2.  **學習時間分析**：根據「上傳時間」，分析我最主要的學習時段（例如：多在週末、或晚上）。
        3.  **內容缺漏與建議**：根據我的「筆記洞察」和「建議行動」，我是否有哪些知識點還不熟悉？或者 AI 建議我做的事情我是否重複在問？
        4.  **總結**：給我一個整體的讀書成效總結與後續學習建議。

        請用 Markdown 格式回覆我，分點條列，標題請加粗。

        [我的知識庫資料]:
        {data_markdown}
        """

        progress(0.6, desc="🤖 正在向 Gemini 2.5 Pro 提出全局分析請求...")
        print("全局分析：正在呼叫 Gemini...")
        model = genai.GenerativeModel('gemini-2.5-pro')
        response = model.generate_content(prompt_text)

        # 步驟 8: 將分析結果寫回 Google Sheet
        print(f"正在將全局分析結果寫回 '{ANALYSIS_WORKSHEET_NAME}' 分頁...")
        try:
            # 確保在此函式中能使用時間套件
            import pytz
            from datetime import datetime as dt

            gsheets = gc.open_by_url(SHEET_URL)

            # 嘗試取得分頁，若不存在則建立
            try:
                wk_analysis = gsheets.worksheet(ANALYSIS_WORKSHEET_NAME)
            except gspread.exceptions.WorksheetNotFound:
                print(f"找不到分頁 '{ANALYSIS_WORKSHEET_NAME}'，將自動建立...")
                wk_analysis = gsheets.add_worksheet(title=ANALYSIS_WORKSHEET_NAME, rows="100", cols="5")
                # 建立新分頁時，寫入標頭
                wk_analysis.append_row(["分析時間", "分析內容"])

            # (可選) 檢查標頭是否真的存在 (處理分頁已存在但為空的情況)
            headers = wk_analysis.get('A1:B1')
            if not headers:
                wk_analysis.append_row(["分析時間", "分析內容"])

            # 準備要寫入的資料 (時間戳 + AI 回應)
            tw_tz = pytz.timezone('Asia/Taipei')
            now_str = dt.now(tw_tz).strftime("%Y-%m-%d %H:%M:%S")
            new_row = [now_str, response.text]

            # 附加新的一列 (Append)
            wk_analysis.append_row(new_row)
            print(f"✅ 成功將分析結果附加至 '{ANALYSIS_WORKSHEET_NAME}' 分頁。")

        except Exception as write_e:
            print(f"❌ 寫回 Google Sheet 時發生錯誤: {write_e}")
            # 即使寫回失敗，Gradio 介面仍應繼續顯示結果

        progress(1.0, desc="✅ 全局分析完成！")
        print("全局分析：已收到 AI 回應。")
        return response.text

    except Exception as e:
        print(f"❌ 全局分析時發生錯誤：{e}")
        return f"❌ 全局分析時發生錯誤：{e}"

# === 3.B 新增：全局 Chatbot 函式 (for 歷史分頁) ===
def handle_global_chat(message, history):
    """
    處理全局 Chatbot 的問答，會讀取「所有」Google Sheet 資料來回答。
    """
    try:
        print("全局 Chatbot：正在載入資料...")
        # 顯示使用者的問題
        history.append((message, None))

        # 1. 載入所有資料
        df = load_knowledge_base()
        if df.empty:
            history[-1] = (message, "❌ 錯誤：知識庫中尚無資料。")
            return "", history

        data_markdown = df.to_markdown(index=False)

        # 2. 準備 Prompt
        prompt_text = f"""
        你是我個人知識庫的 AI 總管。
        請根據我「所有」的歷史筆記資料（Markdown 格式），來回答我的問題。

        [我的知識庫資料]:
        {data_markdown}

        [我的問題]:
        {message}
        """

        # 3. 呼叫 AI
        print("全局 Chatbot：正在呼叫 Gemini...")
        model = genai.GenerativeModel('gemini-2.5-pro')
        response = model.generate_content(prompt_text)

        # 4. 更新 Chatbot 歷史
        history[-1] = (message, response.text)

        return "", history

    except Exception as e:
        print(f"❌ 全局 Chatbot 處理時發生錯誤：{e}")
        history.append((message, f"❌ 處理時發生錯誤：{e}"))
        return "", history

In [73]:
# === 4. 建立 Gradio 介面 ===
print("正在建立 Gradio 介面...")
with gr.Blocks(title="AI 簡報筆記知識庫") as demo:
    gr.Markdown("# 🧠 AI 簡報筆記知識庫")

    # 儲存目前正在分析的檔案名稱 (for Chatbot)
    gemini_file_name_state = gr.State(value=None)

    with gr.Row():
        # --- 左側：上傳區 ---
        with gr.Column(scale=1):
            file_uploader = gr.File(
                label="上傳您的筆記簡報 PDF",
                file_types=[".pdf"]
            )
            submit_btn = gr.Button("🚀 開始分析並存入知識庫", variant="primary")
            # 狀態元件
            status_output = gr.Markdown("請上傳檔案...")

        # --- 右側：雙分頁結果區 ---
        with gr.Column(scale=3):
            with gr.Tabs():
                # --- 分頁 1: 新資料分析 ---
                with gr.Tab("新資料分析"):
                    gr.Markdown("AI 會將分析結果顯示在下方。您也可以在最下方針對此檔案提問。")

                    with gr.Tabs():

                      with gr.Tab("課程簡報摘要"):
                          gr.Markdown("## 📚 課程名稱")
                          md_course_name = gr.Markdown(label="課程名稱", value="#### 尚未分析")
                          gr.Markdown("---")
                          gr.Markdown("## 🗂️ 簡報主題")
                          md_topic = gr.Markdown(label="簡報主題", value="#### 尚未分析")
                          gr.Markdown("---")
                          gr.Markdown("## 📄 簡報內容摘要")
                          md_slide_summary = gr.Markdown(label="簡報內容摘要", value="#### 尚未分析")

                      with gr.Tab("我的筆記洞察"):
                          gr.Markdown("## ✏️ 筆記摘要")
                          md_note_summary = gr.Markdown(label="筆記摘要", value="#### 尚未分析")
                          gr.Markdown("---")
                          gr.Markdown("## 🔍 筆記洞察")
                          md_insights = gr.Markdown(label="筆記洞察", value="#### 尚未分析")
                          gr.Markdown("---")
                          gr.Markdown("## 🚀 建議行動")
                          md_actions = gr.Markdown(label="建議行動", value="#### 尚未分析")

                      with gr.Tab("AI 提問"):
                          gr.Markdown("#### 🤖 針對此檔案提問")
                          chatbot = gr.Chatbot(label="AI 助理", height=300)
                          chat_history = gr.State(value=[])
                          chat_msg = gr.Textbox(label="輸入您的問題...", placeholder="例如：請幫我詳細說明「筆記洞察」中的第二點...")
                          chat_btn = gr.Button("傳送")

                # --- 分頁 2: 歷史知識庫 ---
                with gr.Tab("歷史知識庫"):
                    gr.Markdown("這裡是您 Google Sheet 中的所有歷史紀錄。")

                    with gr.Tabs():

                        # --- 子分頁 1: 歷史資料表格 ---
                        with gr.Tab("資料總表"):
                            # 將 'sheet_output' 的定義移到這裡
                            sheet_output = gr.DataFrame(
                                label="📚 我的 AI 知識庫 (即時更新)",
                                interactive=False,
                                wrap=True
                            )

                        # --- 子分頁 2: Gemini 全局分析 ---
                        with gr.Tab("Gemini 全局分析"):
                            gr.Markdown("點擊下方按鈕，AI 將讀取您「所有」的歷史資料，進行學習成效、時間、內容缺漏等分析。")
                            analyze_btn = gr.Button("🚀 開始進行全局學習分析", variant="primary")
                            gr.Markdown("---")
                            # 新增一個 Markdown 元件來顯示分析結果
                            md_global_analysis = gr.Markdown(label="AI 全局分析報告", value="*尚未分析*")

                        # --- 子分頁 3: AI 知識庫總管 ---
                        with gr.Tab("AI 知識庫總管"):
                            gr.Markdown("#### 🤖 針對「所有」歷史筆記提問")
                            # 注意：使用全新的變數名稱 (global_chatbot) 避免衝突
                            global_chatbot = gr.Chatbot(label="AI 知識庫總管", height=400)
                            global_chat_history = gr.State(value=[])
                            global_chat_msg = gr.Textbox(label="輸入您的問題...", placeholder="例如：我過去所有關於「機器學習」的筆記洞察有哪些？")
                            global_chat_btn = gr.Button("傳送 (全局搜尋)")

    # === 5. 綁定 Gradio 事件 ===

    # [主按鈕] 點擊 "開始分析"
    submit_btn.click(
        fn=process_new_note,
        inputs=[file_uploader, gemini_file_name_state],
        outputs=[
            status_output,
            md_course_name,
            md_topic,
            md_slide_summary,
            md_note_summary,
            md_insights,
            md_actions,
            chatbot, # 清空 chatbot 顯示
            chat_history, # 清空 chatbot 歷史
            gemini_file_name_state, # 儲存新檔案的 name
            sheet_output # 更新歷史表格
        ]
    )

    # [Chatbot] 點擊 "傳送" 或 按下 Enter
    chat_msg.submit(
        fn=handle_chat,
        inputs=[chat_msg, chat_history, gemini_file_name_state],
        outputs=[chat_msg, chatbot] # 清空輸入框，更新 chatbot 顯示
    )
    chat_btn.click(
        fn=handle_chat,
        inputs=[chat_msg, chat_history, gemini_file_name_state],
        outputs=[chat_msg, chatbot] # 清空輸入框，更新 chatbot 顯示
    )

    # [全局分析按鈕] 點擊 "開始進行全局學習分析"
    analyze_btn.click(
        fn=analyze_knowledge_base,
        inputs=None, # 此按鈕不需要輸入
        outputs=[md_global_analysis] # 將結果輸出到 Markdown 元件
    )

    # [全局 Chatbot] 點擊 "傳送" 或 按下 Enter
    global_chat_msg.submit(
        fn=handle_global_chat,
        inputs=[global_chat_msg, global_chat_history],
        outputs=[global_chat_msg, global_chatbot] # 清空輸入框，更新 Chatbot
    )
    global_chat_btn.click(
        fn=handle_global_chat,
        inputs=[global_chat_msg, global_chat_history],
        outputs=[global_chat_msg, global_chatbot]
    )

    # [介面載入時] 自動載入歷史資料
    demo.load(
        fn=load_knowledge_base,
        inputs=None,
        outputs=[sheet_output]
    )

正在建立 Gradio 介面...


/tmp/ipython-input-2389066004.py:51: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="AI 助理", height=300)
/tmp/ipython-input-2389066004.py:83: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  global_chatbot = gr.Chatbot(label="AI 知識庫總管", height=400)


In [74]:
# === 6. 啟動 Gradio ===
print("Gradio 介面建立完成，正在啟動...")
demo.launch(share=True, debug=True)

Gradio 介面建立完成，正在啟動...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7adfff27c3a1a88a53.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


正在從 Google Sheet 載入資料...
✅ 成功載入 5 筆資料。
正在從 Google Sheet 載入資料...
✅ 成功載入 5 筆資料。
全局分析：正在載入資料...
正在從 Google Sheet 載入資料...
✅ 成功載入 5 筆資料。
全局分析：正在呼叫 Gemini...


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

正在將全局分析結果寫回 'Gemini 全局分析' 分頁...
✅ 成功將分析結果附加至 'Gemini 全局分析' 分頁。
全局分析：已收到 AI 回應。
正在處理檔案: '/tmp/gradio/a5bc29b016896fd02396e92ac167b3ded0e051df985c5e16c30248dc3ec510e7/第六章 意識 ppt_251105_175714.pdf'
✅ 檔案上傳成功！(File Name: files/izdpyio8tw5v)
✅ AI 分析完成，已解析 JSON。
正在從 Google Sheet 載入資料...
✅ 成功載入 5 筆資料。
✅ 已將新資料附加到 Google Sheet。
正在從 Google Sheet 載入資料...
✅ 成功載入 6 筆資料。
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> None
Killing tunnel 127.0.0.1:7861 <> https://7adfff27c3a1a88a53.gradio.live
